# AIM

Simultaneous Statr Transfer

In [1]:
import sys 
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram')
sys.path.append('C:\\Users\\Eesh Gupta\\Documents\\RU Research\\Chakram\\ECD_control')

# For Prof. Chakram Desktop 
# sys.path.append('C:\\_Lib\\python\\ECD_control')
# sys.path.append('C:\\_Lib\\python')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#from  ECD_control import ECD_optimization  
#from ECD_control import ECD_pulse_construction

import numpy as np
import qutip as qt 
from DECD_param import FixedBatchOptimizer as BatchOptimizer
#from ECD_control.ECD_optimization.batch_optimizer import BatchOptimizer # contains custom initial angles framework
#from ECD_control.ECD_optimization.optimization_analysis import OptimizationAnalysis, OptimizationSweepsAnalysis
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-08-22 09:14:31.845443: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16058308920640112601
 xla_global_id: -1]

In [20]:
#Modes Truncation
N1 = 3
N2 = 3

### Batch Optimizer

In [52]:
def get_params(n_layers, initial, target): 
    '''
    Runs Batch Optimizer to produce a unitary which maps given initial state to final state
    
    Note: It solves a State Transfer Problem
    
    Returns parameters beta, gamma, phi and theta as a function of layers
    '''
    
    #Optimization of ECD Circuit parameters (betas, phis, and thetas)
    #the optimization options
    opt_params = {
    'N_blocks' : n_layers, #circuit depth
    'N_multistart' : 10, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 200, #number of epochs before termination
    'epoch_size' : 20, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 0.995, #terminal fidelitiy
    'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
    'beta_scale' : 3.0, #maximum |beta| for random initialization
    'gamma_scale' : 3.0, #maximum |gamma| for random initialization
    'N_cav1': N1, #number of levels in mode 1
    'N_cav2': N2, #number of levels in mode 2
    'initial_states' : initial, #qubit tensor oscillator, start in |g> |0>
    'target_states' : target, #end in |e> |target>.
    #"initial_params": init_params,
    'name' : 'Fock1 %d' % Fock1, #name for printing and saving
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
    }



    #note: optimizer includes pi pulse in every ECD step. However, final ECD step is implemented 
    #in experiment as a displacement since the qubit and oscillator should be disentangled at this point.
    #So, we ask the optimizer to end in |e> |target> instead of |g>|target>.
    
    #create optimization object. 
    #initial params will be randomized upon creation
    opt = BatchOptimizer(**opt_params)

    #print optimization info. 
    opt.print_info()
    
    
    #run optimizer.
    #opt.optimize()
    
    #get optimized param
    reported_fid = opt.best_circuit()['fidelity']
    betas = opt.best_circuit()['betas']
    gammas = opt.best_circuit()['gammas']
    phis = opt.best_circuit()['phis']
    thetas = opt.best_circuit()['thetas']

    return reported_fid, betas, gammas, phis, thetas
    

### Get the Pulses

In [8]:
#from ECD_control import ECD_optimization
#from ECD_control import ECD_pulse_construction

# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()
#from DECD_pulse import *

In [9]:
# def alpha_from_epsilon_nonlinear_finite_difference(
#     epsilon_array, delta=0, alpha_init=0 + 0j):
#     '''
#     These are equations of motion used in cancelling d alpha/dt and cavity drive terms
#     '''
#     dt = 1
#     alpha = np.zeros_like(epsilon_array)
#     alpha[0] = alpha_init
#     alpha[1] = alpha_init
#     for j in range(1, len(epsilon_array) - 1):
#         alpha[j + 1] = (
#                 2*
#                  dt
#                 * (
#                     -1j * delta# * alpha[j]
#                     #- 2j * Ks * np.abs(alpha[j]) ** 2 * alpha[j]
#                     #- (kappa / 2.0) * alpha[j]
#                     - 1j * epsilon_array[j]
#                 )
#                 + alpha[j - 1])
#     return alpha

# def alpha_star_from_epsilon_nonlinear_finite_difference(
#     epsilon_array, delta=0, alpha_init=0 + 0j):
#     dt = 1
#     alpha = np.zeros_like(epsilon_array)
#     alpha[0] = alpha_init
#     alpha[1] = alpha_init
#     for j in range(1, len(epsilon_array) - 1):
#         alpha[j + 1] = (
#                 2*
#                  dt
#                 * (
#                     1j * delta# * alpha[j]
#                     #- 2j * Ks * np.abs(alpha[j]) ** 2 * alpha[j]
#                     #- (kappa / 2.0) * alpha[j]
#                     + 1j * epsilon_array[j]
#                 )
#                 + alpha[j - 1])
#     return alpha

In [10]:
# def get_pulses(betas, gammas, phis, thetas):
#     '''
#     Returns alpha(t) (displacements) and qubit pulse for the Batch Optimizer's generated parameters
#     '''
    
#     #Our cavity(ies) and qubit. When using pulse construction on the FPGA, these objects can be loaded, and 
#     #Here, we are making a "FakeCavity" and a "FakeQubit" with parameters found from experimental calibrations.

#     storage1_params = {
#             "chi_kHz": -33, #dispersive shift
#             "chi_prime_Hz": 0, #second order dispersive shift
#             "Ks_Hz": 0, #Kerr correction not yet implemented.
#             "epsilon_m_MHz": 400, #largest oscillator drive amplitude in MHz (max|epsilon|)
#             "unit_amp": 0.01, #DAC unit amp of gaussian displacement to alpha=1.
#             "sigma": 11, #oscillator displacement sigma
#             "chop": 4, #oscillator displacement chop (number of stds. to include in gaussian pulse)
#         }
#     storage2_params = {
#             "chi_kHz": -33, #dispersive shift
#             "chi_prime_Hz": 0, #second order dispersive shift
#             "Ks_Hz": 0, #Kerr correction not yet implemented.
#             "epsilon_m_MHz": 400, #largest oscillator drive amplitude in MHz (max|epsilon|)
#             "unit_amp": 0.01, #DAC unit amp of gaussian displacement to alpha=1.
#             "sigma": 11, #oscillator displacement sigma
#             "chop": 4, #oscillator displacement chop (number of stds. to include in gaussian pulse)
#         }
#     qubit_params = {'unit_amp': 0.5, 'sigma': 6, 'chop': 4} #parameters for qubit pi pulse.

#     #calculating conversion between qubit DAC units and MHz (Omega)
#     pi = rotate(np.pi, phi=0, sigma=qubit_params['sigma'], chop=qubit_params['chop'], dt=1)
#     Omega_m = np.real(np.max(pi))/qubit_params['unit_amp']

#     #creating the "Fake storage" and "Fake qubit"
#     storage1 = FakeStorage(**storage1_params)
#     storage2 = FakeStorage(**storage2_params)
#     qubit = FakeQubit(**qubit_params)

#     #The maximum displacement used during the ECD gates.
#     alpha_CD1 = 30
#     alpha_CD2 = 30
#     #Small delay to insert between oscillator and qubit pulses
#     buffer_time = 4
    
    
#     #generating the conditional displacement circuit.
#     #set 'chi_prime_correction = True' to correct for linear part of chi'
#     #final_disp = True will implement final ECD gate as a displacement
#     pulse_dict = conditional_displacement_circuit(betas, gammas, phis, thetas, storage1, storage2, qubit,
#                          alpha_CD1, alpha_CD2 ,buffer_time=buffer_time, kerr_correction = False, chi_prime_correction=True, final_disp=True, pad=True)
#     cavity1_dac_pulse, cavity2_dac_pulse, qubit_dac_pulse, = pulse_dict['cavity1_dac_pulse'], pulse_dict['cavity2_dac_pulse'], pulse_dict['qubit_dac_pulse']
    
#     #Dac units to Ghz conversion

#     cavity1_dac_pulse_GHz = (2*np.pi)*(10**(-3))*storage1.epsilon_m_MHz*cavity1_dac_pulse #convert from DAC to Mhz to Ghz
#     cavity2_dac_pulse_GHz = (2*np.pi)*(10**(-3))*storage2.epsilon_m_MHz*cavity2_dac_pulse #convert from DAC to Mhz to Ghz
#     qubit_dac_pulse_GHz = (2*np.pi)*10**(-3)*qubit.Omega_m_MHz*qubit_dac_pulse #convert from DAC to Mhz to Ghz

#     alpha1 = alpha_from_epsilon_nonlinear_finite_difference(
#         epsilon_array =cavity1_dac_pulse_GHz , delta=0, alpha_init=0 + 0j)
#     alpha1_star = alpha_star_from_epsilon_nonlinear_finite_difference(
#         epsilon_array =np.conjugate(cavity1_dac_pulse_GHz) , delta=0, alpha_init=0 + 0j)
    
#     alpha2 = alpha_from_epsilon_nonlinear_finite_difference(
#         epsilon_array =cavity2_dac_pulse_GHz , delta=0, alpha_init=0 + 0j)
#     alpha2_star = alpha_star_from_epsilon_nonlinear_finite_difference(
#         epsilon_array =np.conjugate(cavity2_dac_pulse_GHz) , delta=0, alpha_init=0 + 0j)
    
#     return alpha1, alpha1_star, alpha2, alpha2_star, qubit_dac_pulse_GHz, storage1_params, storage2_params

### Qutip Simulation

In [11]:
# #for time dep hammy
# def mult(a,b):
#     '''
#     Elementwise multiplication of two lists
#     '''
#     if len(a) != len(b):
#         print('Error with mult: lists must be of same length')
#         return []
#     output = [a[i]*b[i] for i in range(len(a)) ]
#     return np.asarray(output) 

# # qutip doesn't like complex numbers in parenthesis notation which is default python notation
# # z = (a + bi)
# # what qutip wants is z = a+ bi
# #This is achieved by the np.asarray() function

def dot(state1, state2):
    '''
    finding prob of measure state2 given we have prepared state 1
    '''
    fid = state1.overlap(state2)
    return fid*np.conjugate(fid)

In [21]:
from qutip import *
from qutip.control import *
from math import *

def get_qutip_sim(alpha1, alpha1_star, alpha2, alpha2_star,
                  qubit_dac_pulse_GHz, storage1_params, storage2_params,
                 sim_params, initial, final):
    '''
    simulation using qutip and return fidelities
    
    sim_params ( a dictionary) turns various terms on and off in full hamiltonaian
    '''
    
    T = len(alpha1)
    times = np.linspace(0, T, T) #1ns timesteps
    
    n_c1 = N1  #number of levels in cavity
    n_c2 = N2  #number of levels in cavity
    n_q = 2 #number of levels in qubit

    #N = 1 #target cavity state

    omega_c1  = 5.26 *2*pi   #frequencies in GHz
    omega_c2 = 5.26 *2*pi   #frequencies in GHz
    omega_q = 6.65*2*pi
    #K = -3.7*(10**(-6))*2*pi #cavity nonlinearity

    #photon depenedent number shifts
    chi1 = storage1_params["chi_kHz"]*2*pi*(10**(-6)) #linear
    chi1_ = storage1_params["chi_prime_Hz"]*(10**(-9)) #quadratic
    chi2 = storage2_params["chi_kHz"]*2*pi*(10**(-6)) #linear
    chi2_ = storage2_params["chi_prime_Hz"]*(10**(-9)) #quadratic

    ##mode to mode coupling 
    eta = 0#10**(-6)
    
    ## Transmon Relaxation and Dephasing (Data from Fast ECD paper)
    T1 = 50*(10**(3))   # 50 microseconds in nanosecond 
    Techo = 65*(10**(3)) 
    gamma_relax = 1/T1
    gamma_echo = 1/Techo
    gamma_phi = gamma_echo - (gamma_relax/2)
    
    # Creating identity, annhilitation/creation, num operators 
    identity_q = qeye(n_q)
    identity_c = qeye(n_c1)

    a_q = destroy(n_q)
    a_c = destroy(n_c1)

    adag_q = create(n_q)
    adag_c = create(n_c1)

    num_q = num(n_q)
    num_c =  num(n_c1)


    c_ops = [
        sim_params['transmon_relax']*tensor(a_q, identity_c, identity_c),
        sim_params['transmon_dephasing']*tensor(num_q, identity_c, identity_c)
    ]

    tlist = times
    
    # Displace Frame hammy
    H0_disp = ((chi1/2)*tensor(sigmaz(), num_c, identity_c) # first mode -qubit
                +
              (chi2/2)*tensor(sigmaz(), identity_c, num_c) # 2nd mode - qubit
               +
              (eta)*tensor(identity_q, num_c, num_c)) # MODE-MODE
    
    # Preparing coefficients of various time dependent terms in Hamiltonian 
    
    alpha1_sq =np.real( [np.real(i)**2 + np.imag(i)**2 for i in alpha1])
    alpha2_sq =np.real( [np.real(i)**2 + np.imag(i)**2 for i in alpha2])
    alpha1_times_2 = mult(alpha1, alpha2)
    alpha1star_times_2 = mult(alpha1_star, alpha2)

    H = [H0_disp,
         #Qubit drive
        [tensor(a_q, identity_c, identity_c), np.conjugate(qubit_dac_pulse_GHz)], 
         [tensor(adag_q, identity_c, identity_c), qubit_dac_pulse_GHz],

         # Desired Displacement term from qubit-mode1 coupling
         [(chi1/2)*tensor(sigmaz(), a_c, identity_c), alpha1_star],  
         [(chi1/2)*tensor(sigmaz(), adag_c, identity_c), alpha1],

         # Desired Displacement term from qubit-mode2 coupling
         [(chi2/2)*tensor(sigmaz(),  identity_c, a_c), alpha2_star],  
         [(chi2/2)*tensor(sigmaz(), identity_c, adag_c), alpha2],
         
         #Stark Shifts
         [sim_params['Stark Shift']*(chi1/2)*tensor(sigmaz(), identity_c, identity_c), alpha1_sq],
         [sim_params['Stark Shift']*(chi2/2)*tensor(sigmaz(), identity_c, identity_c), alpha2_sq],
        
         # UnDesired Displacement term from mode-mode coupling (no disp^2)

         [sim_params['Mode Coupling']*(eta)*tensor(identity_q, a_c, num_c), alpha1_star],  
         [sim_params['Mode Coupling']*(eta)*tensor(identity_q, adag_c, num_c), alpha1],
         [sim_params['Mode Coupling']*(eta)*tensor(identity_q, num_c, a_c), alpha2_star],  
         [sim_params['Mode Coupling']*(eta)*tensor(identity_q, num_c, adag_c), alpha2],

         [sim_params['Mode Coupling']*(eta)*tensor(identity_q, a_c, a_c), np.conjugate(alpha1_times_2)],  
         [sim_params['Mode Coupling']*(eta)*tensor(identity_q, a_c, adag_c), alpha1star_times_2],
         [sim_params['Mode Coupling']*(eta)*tensor(identity_q, adag_c, a_c), np.conjugate(alpha1star_times_2)],  
         [sim_params['Mode Coupling']*(eta)*tensor(identity_q, adag_c, adag_c), alpha1_times_2],

         [sim_params['Mode Coupling']*(eta)*tensor(identity_q, identity_c, num_c), alpha1_sq],
         [sim_params['Mode Coupling']*(eta)*tensor(identity_q, identity_c, num_c), alpha2_sq],
     
        ]
        
        
        
        
    
    print('starting evolution')
    psi0 = tensor(basis(n_q,0), initial)
    opts = Options(store_states=True, store_final_state=True,nsteps = 4000 ,max_step = 10)
    output = mesolve(H, psi0 , times, c_ops, [], options =opts)
    #rho = output.states[-1]
    print('finished evolution')
    
#     target_g = tensor(basis(n_q,0), target)
#     fid1 = dot(target_g, rho)
    
#     target_e = tensor(basis(n_q,1 ), basis(n_c, 1))
#     fid2 = dot(target_e, rho)
    
    return output


In [22]:
# sim_params = {'Stark Shift': 0, 'Mode Coupling': 0}  # 0 means false, 1 means true or activates
# ecd_fid = get_qutip_sim(alpha1, alpha1_star, alpha2, alpha2_star,
#           qubit_dac_pulse_GHz, storage1_params, storage2_params, 
#                          sim_params)
# ecd_terms.append(ecd_fid)

### Calculator (Not Needed)
Since Batch Optimizer agrees generally with calc

In [23]:
from qutip import *

In [24]:
def mod_disp_op(disp, n_q, n_c):
    '''
    Returns displacement operator using baken campbell formula
    '''
    pauli_like_x = (create(n_c) + destroy(n_c))
    pauli_like_y = (1j)*(create(n_c) - destroy(n_c))
    comm = (1/2)*((pauli_like_x*pauli_like_y) - (pauli_like_y*pauli_like_x))
    re = np.real(disp)
    im = np.imag(disp)

    first = (1j*im*pauli_like_x).expm()
    second = (-1j*re*pauli_like_y).expm()
    third = ((im*re)*(-1)*comm).expm()
    return first*second*third

def disp_op(disp, n_q, n_c):
    '''
    Returns displacement operator for specified displacement
    '''
    #disp = normalize_complex(disp)
    exponent = (disp*create(n_c))- (np.conjugate(disp)*destroy(n_c))
    return  exponent.expm()

def cond_disp_op(beta, n_q, n_c1, n_c2,mode, use_mod):
    '''
    Returns cond displacement operator for specified real displacement
    '''
    disp = beta/2
    
    n_c = n_c1 if mode == 1 else n_c2
    
    if use_mod: 
        d = mod_disp_op(disp= disp, n_q = n_q, n_c = n_c) #Baker Campbell Approx
    else: 
        d = disp_op(disp= disp, n_q = n_q, n_c = n_c)
    d_adjoint = d.dag()
    
    if mode == 1: 
        left = tensor(create(n_q), d, qeye(n_c2)) #D(beta/2)|e><g|
        right = tensor(destroy(n_q), d_adjoint, qeye(n_c2)) #D(-beta/2)|g><e|   ...not sure d_adjoint(alpha) = d(-alpha) if use Baker
    
    else: #mode ==2 
        left = tensor(create(n_q), qeye(n_c2), d)
        right = tensor(destroy(n_q),  qeye(n_c2), d_adjoint)
    return left+right

def qubit_rot(phi, theta, n_q, n_c1, n_c2):
    '''
    Returns qubit rotation
    '''
    rot = (np.cos(phi)*sigmax()) + (np.sin(phi)*sigmay())
    exp = (-1.0j)*(theta/2)*(rot)
    return tensor(exp.expm(), qeye(n_c1), qeye(n_c2) )

def normalize_complex(number):
    '''
    Returns radius r of complex number z = r*e^iphi
    '''
    return np.sqrt(number.real**2 + (number .imag**2))

def dot(state1, state2):
    '''
    dotting both states
    '''
    fid = state1.overlap(state2)
    return fid*np.conjugate(fid)
def evolve(initial_state, betas, gammas, phis, thetas, n_q, n_c1, n_c2, use_mod = False):
    '''
    Operates on initial_state with ECD(beta_n)*R(phi_n, theta_n) *...........*ECD(beta_1)*R(phi_1, theta_1)
    '''
    n = len(betas)
    state = initial_state
    for i in range(n):
        beta = betas[i]
        gamma = gammas[i]
        phi = phis[i]
        theta = thetas[i]
        
  
    
        #state = cond_disp_op(gamma, n_q, n_c1,n_c2,mode = 2,use_mod = use_mod)*cond_disp_op(beta, n_q, n_c1,n_c2, mode = 1,use_mod = use_mod)*qubit_rot(phi, theta, n_q, n_c1, n_c2)*state
        state = cond_disp_op(beta, n_q, n_c1,n_c2, mode = 1,use_mod = use_mod)*cond_disp_op(gamma, n_q, n_c1,n_c2,mode = 2,use_mod = use_mod)*qubit_rot(phi, theta, n_q, n_c1, n_c2)*state
           
    return state
        
        

In [25]:
# initial = tensor(basis(2, 0), basis(N1, 0), basis(N2, 0))
# target = tensor(basis(2, 0), basis(N1, 1), basis(N2, 0))
# pi_target = tensor(basis(2, 1), basis(N1, 1), basis(N2, 0))

In [54]:
def Calculator(betas,gammas, phis, thetas, initial, target): 
    '''
    Returns fidelity using qutip matrix algebra
    '''
    fids = []
    for k in range(len(initial)):
        init = initial[k]
        tar = target[k]
        
        rho_calc = evolve(init,betas,gammas, phis, thetas,n_q = 2,n_c1 = N1, n_c2 = N2,
                       use_mod = False)
        fid = dot(tar, rho_calc)
        fids.append(fid)
    return np.mean(fids)

In [40]:
#Calculator(betas,gammas, phis, thetas)

# Main
Vary conditions, see differences in fidelities

In [41]:
#Initial oscillator state.
Fock1 = 0
Fock2= 1
psi_i1 = qt.basis(N1,Fock1) #initial state
psi_i2 = qt.basis(N2,Fock2)
initial1 = qt.tensor(basis(2,0), psi_i1, psi_i2)

#Target
Fock1 = 1
Fock2= 0
psi_t1 = qt.basis(N1,Fock1) #target state
psi_t2 = qt.basis(N2,Fock2)
target1 = qt.tensor(basis(2,0), psi_t1, psi_t2)
#-------------------------------------------------------
#Second state 
#Initial oscillator state.
Fock1 = 0
Fock2= 2
psi_i1 = qt.basis(N1,Fock1) #initial state
psi_i2 = qt.basis(N2,Fock2)
initial2 = qt.tensor(basis(2,0), psi_i1, psi_i2)


#Target
Fock1 = 2
Fock2= 0
psi_t1 = qt.basis(N1,Fock1) #target state
psi_t2 = qt.basis(N2,Fock2)
target2 = qt.tensor(basis(2,0), psi_t1, psi_t2)

#-------------------------FUll list
initial = [initial1, initial2]
target = [target1, target2]

In [42]:
#BO_fid, betas, gammas, phis, thetas = get_params(12, initial, target)

Get parameters from a file

In [43]:
fname = '01_02_to_10_20_Aug22_num_layers_params.csv'

In [44]:
import pandas as pd

In [45]:
columns=['layers', 'exact fidelity','bo_fidelity', 'betas_r', 'betas_i', 'gammas_r', 'gammas_i', 'phis', 'thetas']
df = pd.DataFrame([[None for i in range(len(columns))]],columns=columns)
df.to_csv(fname, index=False)
df

,layers,exact fidelity,bo_fidelity,betas_r,betas_i,gammas_r,gammas_i,phis,thetas
0,None,None,None,None,None,None,None,None,None


In [46]:
#BO_fid, betas, gammas, phis, thetas = get_params(2, initial, target)

In [47]:
#list(np.real(list(betas)))

In [56]:
import pandas as pd
from IPython.display import clear_output

def main(times ,layers): 
    '''
    Get param, find exact fidelity and store
    '''
    df = pd.read_csv(fname)
    
    for l in layers:
    
        for i in range(times): 
            
            BO_fid, betas, gammas, phis, thetas = get_params(l, initial, target)
            c_fid = np.real(Calculator(betas,gammas, phis, thetas, initial, target))
            df_new = pd.DataFrame([[l, c_fid, BO_fid, list(np.real(betas)),list(np.imag(list(betas))),list(np.real(list(gammas))),list(np.imag(list(gammas))), list(phis), list(thetas)]], columns=columns)
            if type(df) ==None : 
                df = df_new
            else:
                df = df.append(df_new, ignore_index = True)
            df.to_csv(fname, index=False)
            clear_output(wait = True)
            print('num_layer: ' + str(l))
            print('sample: ' + str(i))
    return df

In [57]:
layers = [2 + 2*i for i in range(10)]
main(5, layers)

num_layer: 10
sample: 1
None
Fixed construct needed matrices called


KeyboardInterrupt: 

In [72]:
#betas

array([ 2.1002693 -0.17033613j, -0.17127126-0.27397758j], dtype=complex64)

In [ ]:
#BO_fid, betas, gammas, phis, thetas = get_params(3, initial, target)

In [ ]:
#alpha1, alpha1_star, alpha2, alpha2_star, qubit_dac_pulse_GHz, storage1_params, storage2_params = get_pulses(betas, gammas, phis, thetas)

In [ ]:
# sim_params = {'Stark Shift': 0, 'Mode Coupling': 0, 'transmon_relax': 0,'transmon_dephasing': 0 }  # 0 means false, 1 means true or activates

# #Run the qutip fids
# print('Only ECD term in Qutip')
# ecd_fid = get_qutip_sim(alpha1, alpha1_star, alpha2, alpha2_star,
#           qubit_dac_pulse_GHz, storage1_params, storage2_params, 
#                          sim_params, initial, target)

In [ ]:
#ecd_fid

In [ ]:
# from IPython.display import clear_output

# BO_fids = [] #Fidelities reported by Batch Optimizer
# #BO_errs = []

# # Fidelities due to 
# ecd_fids = []     #only conditional displacement and dispersive shift term
# #ecd_errs = []

# ss_fids = []      # Adding stark shift terms
# #ss_errs = []

# mm_fids = []      # Mode mode coupling terms
# #mm_errs = [] 

# tr_fids = [] # Transmon relaxation

# tp_fids = [] # Transmon dephasing

# N_times = 1     # No. of optimized sets of parameters


# from IPython.display import clear_output
    
# for k in range(N_times): 
    
#     #First get Batch OPt fids
#     #BO_fid, betas, gammas, phis, thetas = get_params(3, initial, target)
#     BO_fids.append(BO_fid)
#     #clear_output()
#     #BO_fids.append(0.995)
    
#     #Then get pulses
#     #alpha1, alpha1_star, alpha2, alpha2_star, qubit_dac_pulse_GHz, storage1_params, storage2_params = get_pulses(betas, gammas, phis, thetas)
#     sim_params = {'Stark Shift': 0, 'Mode Coupling': 0, 'transmon_relax': 0,'transmon_dephasing': 0 }  # 0 means false, 1 means true or activates
    
#     #Run the qutip fids
#     print('Only ECD term in Qutip')
#     ecd_fid = get_qutip_sim(alpha1, alpha1_star, alpha2, alpha2_star,
#               qubit_dac_pulse_GHz, storage1_params, storage2_params, 
#                              sim_params, initial, target)
#     ecd_fids.append(ecd_fid)
    
#     print('Now adding Stark Shifts')
#     sim_params['Stark Shift'] = 1
#     ss_fid = get_qutip_sim(alpha1, alpha1_star, alpha2, alpha2_star,
#               qubit_dac_pulse_GHz, storage1_params, storage2_params, 
#                              sim_params, initial, target)
#     ss_fids.append(ss_fid)
    
#     print('Now adding Mode Mode Coupling')
#     sim_params['Mode Coupling'] = 1
#     mm_fid = get_qutip_sim(alpha1, alpha1_star, alpha2, alpha2_star,
#               qubit_dac_pulse_GHz, storage1_params, storage2_params, 
#                              sim_params, initial, target)
#     mm_fids.append(mm_fid)
    
    
#     print('Now adding Transmon Relaxation')
#     sim_params['transmon_relax'] = 1
#     tr_fid = get_qutip_sim(alpha1, alpha1_star, alpha2, alpha2_star,
#               qubit_dac_pulse_GHz, storage1_params, storage2_params, 
#                              sim_params, initial, target)
#     tr_fids.append(tr_fid)
    
#     print('Now adding Transmon Dephasing')
#     sim_params['transmon_dephasing'] = 1
#     tp_fid = get_qutip_sim(alpha1, alpha1_star, alpha2, alpha2_star,
#               qubit_dac_pulse_GHz, storage1_params, storage2_params, 
#                              sim_params, initial, target)
#     tp_fids.append(tp_fid)
    
    
#     clear_output(wait = True)
    



In [ ]:
# BO_fids = [0.9,0.8]
# mm_fids = BO_fids
# ss_fids = BO_fids
# ecd_fids = BO_fids

In [ ]:
# import matplotlib.pyplot as plt
# plt.rcParams['font.size'] = 20

# #Interpreting Results
# conditions = ['Batch Optimizer','ECD Unitary','Stark Shift', 'Mode-Mode Coupling', 'Transmon Relaxation', 'Transmon Dephasing']
# x_pos = np.arange(len(conditions))

# results = [BO_fids, ecd_fids, ss_fids, mm_fids, tr_fids, tp_fids]
# means = [1- np.mean(fids) for fids in results]
# errs = [np.std(fids) for fids in results ]

# # Build the plot
# fig, ax = plt.subplots(figsize = (8,6))
# ax.bar(x_pos, means, yerr=errs, align='center', alpha=0.5, ecolor='black', capsize=10)
# ax.set_ylabel('$1-\mathcal{F}$')
# ax.set_xticks(x_pos)
# ax.set_xticklabels(conditions, fontsize = 15)
# ax.set_title('Decoherence Free Error Budget')
# ax.yaxis.grid(True)

# # Save the figure and show
# plt.tight_layout()
# #plt.savefig('Decoherence Free Error Budget.png')
# plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# plt.rcParams['font.size'] = 12
# figure = plt.figure(figsize= (8, 6))
# # plt.plot(layers, qutip_diffs, marker = 'o', linestyle = '--',label = 'Qutip')
# # plt.plot(layers, BO_diffs, marker = 'o',label = 'Batch Optimizer (Using BCH approx)')
# plt.errorbar(layers, qutip_diffs, yerr = qutip_errs,capsize=10,  fmt ='o', label = 'Qutip')
# plt.errorbar(layers, BO_diffs, yerr = BO_errs,capsize=10, fmt ='o', label = 'Batch Optimizer (Using BCH approx)')

# #plt.plot(layers, rep_fids, marker = 'o', linestyle = '--',label = 'True Fidelity' )
# plt.xlabel('Layers')
# plt.ylabel(' Error from Exact Fidelity')
# plt.legend()

In [ ]:
# def diff_arrays(a,b):
#     '''
#     Input : two arrays ; a = [a1, a2, ..., aN], b = [b1, b2, ..., bN]
#     Output : c = [a1 -b1, a2 - b2, .....]
#     '''
#     c = [a[i]- b[i] for i in range(len(a))]
#     return c